In [421]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_score, accuracy_score
savePath = 'extract/'

In [394]:
targetPeriod="2018 - 01"
CurrentYear = 2018

# create a list of all periods
allPeriods = []
for i in range (1,14):
     allPeriods.append(str(CurrentYear)+ ' - ' +  str("%02d" % (i,)))
futurePeriods = [i for i in allPeriods if i >= targetPeriod]


In [456]:
# Generate report files
for i in range(1,12):   
    file = "FY18-P"+str(i)
    data = pd.read_excel("../../History/processed/"+file+".xlsx")
    # Cleaning data
    data = data[[
           'Total Est. Engagement Revenue', 
           'Current Status', 'Sales Stage',
           'Market Offering Solution', 'Market Offering', "Close Period",
           'Industry', 
           'Market']]
    data = data[data['Market Offering Solution'] == "Technology"]
    data['Current Status'].fillna("nothing", inplace=True)
    data['OSL']=data['Current Status'].str.contains('OSL')
    data['OSL'].replace([False,True],[0,1],inplace=True)
    data.drop("Current Status", axis=1, inplace=True)
    data.drop("Market Offering Solution", axis=1, inplace=True)
    sl = pd.get_dummies(data["Market Offering"], drop_first=True)
    ind = pd.get_dummies(data["Industry"], drop_first=True)
    mark = pd.get_dummies(data["Market"], drop_first=True)
    data.drop("Market Offering", axis=1, inplace=True)
    data.drop("Industry", axis=1, inplace=True)
    data.drop("Market", axis=1, inplace=True)
    data = pd.concat([data, sl, ind, mark],axis=1)
    # extracting x & y for training data (closed deals) and standardizing
    closeDeals = data[data['Sales Stage'].str.contains("Closed|Lost|Sold")].copy()
    closeDeals.drop("Close Period", axis=1, inplace = True)
    closeDeals['Sales Stage'].replace(["06 - Closed - Sold/Fermer - Vendu", "Sold", "07 - Closed - Lost/Fermer - Perdu", "08 - Closed - Abandoned/Fermer - Abandonné","Lost"],[1,1,0,0,0],inplace=True)
    x= closeDeals.drop('Sales Stage', axis=1)
    y= closeDeals['Sales Stage']
    scaler = StandardScaler()
    scaler.fit(x)
    x = scaler.transform(x)
    # fitting and predicting model on each period, multiple fittings are done to get average
    allResults = []
    allSolds = []
    allPipe = []
    allPipeNotLost = []
    for period in futurePeriods:
        preddata = data[data['Close Period']==period].copy()
        alreadySold = preddata[preddata['Sales Stage'].isin(["06 - Closed - Sold/Fermer - Vendu",'Sold'])].sum()['Total Est. Engagement Revenue']
        allSolds.append(alreadySold)
        totalPipe = preddata.sum()['Total Est. Engagement Revenue']
        allPipe.append(totalPipe)
        totalPipeNotLost = preddata[~preddata['Sales Stage'].isin(["07 - Closed - Lost/Fermer - Perdu", "08 - Closed - Abandoned/Fermer - Abandonné",'Lost'])].sum()['Total Est. Engagement Revenue']
        allPipeNotLost.append(totalPipeNotLost)
        preddata = preddata[~preddata['Sales Stage'].str.contains('Closed|Lost|Sold')]
        preddata = preddata.drop('Sales Stage', axis=1)
        preddata = preddata.drop("Close Period", axis=1)
        if (preddata.shape[0]>0):
            predscaled = scaler.transform(preddata)
            result =[]
            for i in range(1,200):
                    rf = RandomForestClassifier()
                    rf.fit(x, y)
                    newpred_prob = rf.predict_proba(predscaled)
                    resultTemp = np.multiply([row[1] for row in newpred_prob] , preddata['Total Est. Engagement Revenue']).sum()
                    result.append(resultTemp)
            result = np.array(result)
            allResults.append(result.mean()+alreadySold)
        else:
            allResults.append(alreadySold)
    allResults = pd.DataFrame(allResults, futurePeriods, columns=['Prediction'])
    allResults['Sold'] = allSolds
    allResults['Pipeline'] = allPipe
    allResults['PipelineNotLost']= allPipeNotLost
    allResults.to_excel(savePath+file+'.xls')
    print("Saved file ", file)



Saved file  FY18-P1
Saved file  FY18-P2
Saved file  FY18-P3
Saved file  FY18-P4
Saved file  FY18-P5
Saved file  FY18-P6
Saved file  FY18-P7
Saved file  FY18-P8
Saved file  FY18-P9
Saved file  FY18-P10
Saved file  FY18-P11


In [228]:
# import matplotlib.pyplot as plt
# target = pd.read_csv("../target.csv")

# # target = target[target['date'].values >=targetPeriod]
# target.set_index('date', inplace=True)
# sns.set_style("whitegrid")
# ax = target.plot(c='r', figsize=(18,8), label="Target")
# print(allResults)

# ax = allResults.plot.bar(ax=ax, label="line 1")
# ax.legend(['Target', 'Forecast'], prop={'size': 16})


In [ ]:
# from os import listdir
# from os.path import isfile, join
# onlyexcel = [f.split('.')[0] for f in listdir(savePath) if f.split('.')[1] == 'xls']
# onlyexcel

In [617]:
predictions = pd.DataFrame()
pipelineTotal = pd.DataFrame()
pipelineNotLost = pd.DataFrame()
allSold = pd.DataFrame()
lastPeriod = 11

for i in range(1,12):
    file = "FY18-P"+str(i)
    data = pd.read_excel(savePath+file+".xls")
    data = data.head(lastPeriod)
    predictions[file] = data['Prediction']
    pipelineTotal[file] = data['Pipeline']
    pipelineNotLost[file] = data['PipelineNotLost']
    allSold[file]= data['Sold']

In [618]:
sold = allSold['FY18-P11']

In [619]:
allSold

,FY18-P1,FY18-P2,FY18-P3,FY18-P4,FY18-P5,FY18-P6,FY18-P7,FY18-P8,FY18-P9,FY18-P10,FY18-P11
2018 - 01,58244858,58244858,58244858,58244858,58244858,58244858,58639709,58244858,58244858,58244858,58244858
2018 - 02,0,23110136,23110136,24153216,24153216,24153216,24586025,24586025,24586025,24586025,24586025
2018 - 03,0,0,44742197,44742197,44742197,44742197,44742197,44742197,44742197,44742197,44742197
2018 - 04,0,0,0,41621829,42337930,42337930,41595638,41256364,41256364,41256364,41256364
2018 - 05,0,0,0,35100,94423642,97583091,97583091,97583091,97583091,97583091,97583091
2018 - 06,0,0,0,0,126005,31800836,32843923,32843923,32843923,33026663,33118663
2018 - 07,0,0,0,0,0,0,75241810,68417313,67346193,67346193,68000489
2018 - 08,0,0,0,0,0,212757,212757,32542666,32599063,33170203,33170203
2018 - 09,0,0,0,0,0,0,0,0,51277945,50994789,50994789
2018 - 10,0,0,0,0,0,0,0,0,0,101162658,103753407


In [554]:
(5.372936e+07-24586025)

29143335.0

In [577]:
pd.DataFrame(actual)

,FY18-P11
2018 - 01,58244858
2018 - 02,24586025
2018 - 03,44742197
2018 - 04,41256364
2018 - 05,97583091
2018 - 06,33118663
2018 - 07,68000489
2018 - 08,33170203
2018 - 09,50994789
2018 - 10,103753407


In [537]:
predictions


,FY18-P1,FY18-P2,FY18-P3,FY18-P4,FY18-P5,FY18-P6,FY18-P7,FY18-P8,FY18-P9,FY18-P10,FY18-P11
2018 - 01,8.348112e+07,5.959855e+07,5.825069e+07,5.824486e+07,5.824486e+07,5.824486e+07,5.899624e+07,5.824486e+07,5.824486e+07,5.824486e+07,5.825044e+07
2018 - 02,5.372936e+07,4.549283e+07,2.509935e+07,2.415322e+07,2.415322e+07,2.415322e+07,2.458602e+07,2.458602e+07,2.458602e+07,2.458602e+07,2.458602e+07
2018 - 03,7.243129e+07,7.455484e+07,6.081315e+07,4.526513e+07,4.474220e+07,4.474220e+07,4.474220e+07,4.474220e+07,4.474220e+07,4.474220e+07,4.474220e+07
2018 - 04,5.705356e+07,6.126025e+07,8.460794e+07,6.721578e+07,4.348721e+07,4.233793e+07,4.159564e+07,4.140724e+07,4.125636e+07,4.125636e+07,4.125636e+07
2018 - 05,2.935126e+07,8.244620e+07,1.002660e+08,8.125728e+07,9.928462e+07,9.765289e+07,9.759669e+07,9.758309e+07,9.758309e+07,9.758309e+07,9.758309e+07
2018 - 06,1.622369e+08,1.560012e+08,7.256135e+07,1.155638e+08,9.582754e+07,4.231080e+07,3.637878e+07,3.332290e+07,3.304055e+07,3.322767e+07,3.331252e+07
2018 - 07,4.135961e+07,4.341102e+07,3.972650e+07,5.572276e+07,9.386706e+07,8.984353e+07,1.236374e+08,8.751595e+07,7.218083e+07,7.150748e+07,7.105477e+07
2018 - 08,8.755754e+07,9.320290e+07,8.655563e+07,1.604695e+08,1.478109e+08,1.459844e+08,1.003667e+08,9.523298e+07,5.263091e+07,4.321667e+07,4.024620e+07
2018 - 09,7.362660e+07,7.422307e+07,9.141652e+07,9.407350e+07,1.069076e+08,9.146291e+07,9.577978e+07,9.374454e+07,7.455719e+07,5.738213e+07,5.382408e+07
2018 - 10,4.565533e+07,4.153844e+07,4.337087e+07,5.693723e+07,3.882593e+07,5.441964e+07,4.280116e+07,7.470471e+07,1.052453e+08,1.258259e+08,1.109204e+08


In [616]:
predictions.subtract(actual,axis=0).divide(actual,axis=0) -0.2

,FY18-P1,FY18-P2,FY18-P3,FY18-P4,FY18-P5,FY18-P6,FY18-P7,FY18-P8,FY18-P9,FY18-P10,FY18-P11
2018 - 01,0.233279,-0.176759,-0.199900,-0.200000,-0.200000,-0.200000,-0.187100,-0.200000,-0.200000,-0.200000,-0.199904
2018 - 02,0.985362,0.650353,-0.179121,-0.217604,-0.217604,-0.217604,-0.200000,-0.200000,-0.200000,-0.200000,-0.200000
2018 - 03,0.418859,0.466321,0.159190,-0.188312,-0.200000,-0.200000,-0.200000,-0.200000,-0.200000,-0.200000,-0.200000
2018 - 04,0.182903,0.284868,0.850785,0.429222,-0.145927,-0.173784,-0.191776,-0.196343,-0.200000,-0.200000,-0.200000
2018 - 05,-0.899218,-0.355118,-0.172506,-0.367302,-0.182563,-0.199285,-0.199861,-0.200000,-0.200000,-0.200000,-0.200000
2018 - 06,3.698656,3.510371,0.990950,2.289385,1.693460,0.077552,-0.101562,-0.193833,-0.202358,-0.196709,-0.194147
2018 - 07,-0.591775,-0.561607,-0.615791,-0.380554,0.180388,0.121219,0.618184,0.086990,-0.138525,-0.148427,-0.155084
2018 - 08,1.439645,1.609838,1.409439,3.637762,3.256134,3.201070,1.825810,1.671040,0.386692,0.102876,0.013324
2018 - 09,0.243806,0.255503,0.592664,0.644767,0.896441,0.593574,0.678227,0.638316,0.262055,-0.074745,-0.144518
2018 - 10,-0.759963,-0.799643,-0.781981,-0.651225,-0.825786,-0.675491,-0.787472,-0.479978,-0.185621,0.012740,-0.130922


In [563]:
pipelineTotal

,FY18-P1,FY18-P2,FY18-P3,FY18-P4,FY18-P5,FY18-P6,FY18-P7,FY18-P8,FY18-P9,FY18-P10,FY18-P11
2018 - 01,335904767,150488131,144816513,144804513,144804513,144804513,145949364,144804513,155304723,155304723,155604723
2018 - 02,132415813,135426703,89926006,87130710,87130710,87130710,88063519,87563519,101650224,101650224,101650224
2018 - 03,179799436,189871006,198188295,125814055,124085749,124085749,124085749,124085749,133720753,133720753,133720753
2018 - 04,140515773,178084543,194865239,237530400,175096334,173027666,172922461,172126100,219336644,219336644,219336644
2018 - 05,108645616,256553627,326436668,208037863,202545903,177385008,174692507,174430979,212637657,212637657,212637657
2018 - 06,437904220,461430009,194798550,256568747,249722537,183926119,165944613,157774811,170594114,170776854,170827075
2018 - 07,106855503,130923685,120615630,152831178,250730070,252879773,271702224,203474806,177940294,166425859,164805356
2018 - 08,264081220,276020502,255818200,524456823,506952229,504545606,261502118,302045243,191373057,181184843,171155426
2018 - 09,173172667,189159299,194431135,220163668,264648243,225870859,319272355,310436967,193477487,159679107,154623793
2018 - 10,98675799,98925799,106925799,125575799,85877176,142124649,128619588,209651020,239502806,259284002,232056846


In [586]:
actual.sum()
sold.sum()

597093084

In [589]:
(predictions.sum().mean() - sold.sum())/sold.sum()

0.3401805565053313

In [588]:
predictions.sum().mean()

800212541.6006045